In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/EMNLP_folder_4

/content/drive/.shortcut-targets-by-id/1ANtRsCBtHNhrdxlObGcNtnE59DOMZyhS/EMNLP_folder_4


In [ ]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install rouge-score -q

     |████████████████████████████████| 2.5MB 4.2MB/s 
     |████████████████████████████████| 3.3MB 43.7MB/s 
     |████████████████████████████████| 901kB 45.2MB/s 
     |████████████████████████████████| 1.2MB 4.2MB/s 


# Importing the Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
#import neptune.new as neptune

from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset)

### Reading the predictions file and test dataset to take its arguments

In [ ]:
path_dataset = '/content/drive/MyDrive/EMNLP_folder_4/dataset/'

In [ ]:
f = open('/content/drive/MyDrive/EMNLP_folder_4/gen_key_points_predictions/6_epochs_training_test_set_generations.txt','r')

preds = f.read().splitlines()

In [ ]:
len(preds)

3923

In [ ]:
df_test = pd.read_csv(path_dataset+'final_test.csv')

In [ ]:
df_test.shape

(3923, 18)

In [ ]:
kp_df = pd.DataFrame({'key_point_id': 3923*['kp_0_10'] ,'key_point':preds,'topic':df_test['topic'],'stance':df_test['stance'],'key_point_given':df_test['key_point'] })

kp_df.head()

,key_point_id,key_point,topic,stance,key_point_given
0,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe..."
1,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights
2,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide
3,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary t...
4,kp_0_10,Parents are not adequately informed about the ...,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe..."


In [ ]:
final_kp_df = kp_df.drop_duplicates()

In [ ]:
final_kp_df.head()

,key_point_id,key_point,topic,stance,key_point_given
0,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe..."
1,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights
2,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide
3,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary t...
4,kp_0_10,Parents are not adequately informed about the ...,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe..."


In [ ]:
final_kp_df.shape

(452, 5)

In [ ]:
final_kp_df['topic'].value_counts()

The USA is a good country to live in                            266
Routine child vaccinations should be mandatory                  156
Social media platforms should be regulated by the government     30
Name: topic, dtype: int64

In [ ]:
df = final_kp_df.groupby("topic")

In [ ]:
df.get_group('Routine child vaccinations should be mandatory')['stance'].value_counts()

-1    96
 1    60
Name: stance, dtype: int64

In [ ]:
df.get_group('Social media platforms should be regulated by the government')['stance'].value_counts()

 1    20
-1    10
Name: stance, dtype: int64

In [ ]:
df.get_group('The USA is a good country to live in')['stance'].value_counts()

 1    154
-1    112
Name: stance, dtype: int64

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

In [ ]:
scores = scorer.score(final_kp_df['key_point'][0],final_kp_df['key_point_given'][0])

In [ ]:
scores['rouge1']

Score(precision=0.0, recall=0.0, fmeasure=0.0)

#### Taking Rouge1 - Precision Metric for selecting the best key_point

In [ ]:
def give_score(kp1,kp_given):
  scores = scorer.score(kp1,kp_given)
  return float(scores['rouge1'][0])

In [ ]:
score_list = []

for i,j in zip(final_kp_df['key_point'],final_kp_df['key_point_given']):
  score_list.append(give_score(i,j))

In [ ]:
final_kp_df['precision_score'] = score_list
final_kp_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,key_point_id,key_point,topic,stance,key_point_given,precision_score
0,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe...",0.000000
1,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights,0.000000
2,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide,0.375000
3,kp_0_10,Not everyone has the ability to go to the doctor,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary t...,0.200000
4,kp_0_10,Parents are not adequately informed about the ...,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe...",0.222222


In [ ]:
df_topic_1 = final_kp_df[final_kp_df['topic'] == 'Routine child vaccinations should be mandatory']
df_topic_2 = final_kp_df[final_kp_df['topic'] == 'Social media platforms should be regulated by the government']
df_topic_3 = final_kp_df[final_kp_df['topic'] == 'The USA is a good country to live in']

In [ ]:
df_topic_1_pos_stance = df_topic_1[df_topic_1['stance'] == 1 ] 
df_topic_1_neg_stance = df_topic_1[df_topic_1['stance'] == -1 ] 
df_topic_2_pos_stance = df_topic_2[df_topic_2['stance'] == 1 ] 
df_topic_2_neg_stance = df_topic_2[df_topic_2['stance'] == -1 ] 
df_topic_3_pos_stance = df_topic_3[df_topic_3['stance'] == 1 ] 
df_topic_3_neg_stance = df_topic_3[df_topic_3['stance'] == -1 ] 

In [ ]:
df_topic_1_pos_stance = df_topic_1_pos_stance.sort_values(by='precision_score', ascending=False)
df_topic_1_neg_stance = df_topic_1_neg_stance.sort_values(by='precision_score', ascending=False)
df_topic_2_pos_stance = df_topic_2_pos_stance.sort_values(by='precision_score', ascending=False) 
df_topic_2_neg_stance = df_topic_2_neg_stance.sort_values(by='precision_score', ascending=False)
df_topic_3_pos_stance = df_topic_3_pos_stance.sort_values(by='precision_score', ascending=False) 
df_topic_3_neg_stance = df_topic_3_neg_stance.sort_values(by='precision_score', ascending=False) 

In [ ]:
df_topic_1_pos_stance.head()

,key_point_id,key_point,topic,stance,key_point_given,precision_score
560,kp_0_10,Child vaccinations are necessary for the prote...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.625
558,kp_0_10,Child vaccinations are necessary for the prote...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,0.600
1074,kp_0_10,"Child vaccines can be given to anyone, but sho...",Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.500
824,kp_0_10,Child vaccines prevent diseases in the child,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.500
695,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.500


In [ ]:
df_topic_1_pos_stance.iloc[:,:]

,key_point_id,key_point,topic,stance,key_point_given,precision_score
560,kp_0_10,Child vaccinations are necessary for the prote...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.625000
558,kp_0_10,Child vaccinations are necessary for the prote...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,0.600000
1074,kp_0_10,"Child vaccines can be given to anyone, but sho...",Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.500000
824,kp_0_10,Child vaccines prevent diseases in the child,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.500000
695,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.500000
694,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.500000
559,kp_0_10,Child vaccinations are necessary for the prote...,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.500000
1076,kp_0_10,"Child vaccines can be given to anyone, but sho...",Routine child vaccinations should be mandatory,1,Routine child vaccinations should be mandatory...,0.454545
696,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Routine child vaccinations should be mandatory...,0.454545
1077,kp_0_10,"Child vaccines can be given to anyone, but sho...",Routine child vaccinations should be mandatory,1,Children should not suffer from preventable di...,0.428571


In [ ]:
keypoints_csv = pd.concat([df_topic_1_pos_stance.iloc[:10,:],df_topic_1_neg_stance.iloc[:10,:],df_topic_2_pos_stance.iloc[:,:],df_topic_2_neg_stance.iloc[:,:],df_topic_3_pos_stance.iloc[:10,:],df_topic_3_neg_stance.iloc[:10,:]])

In [ ]:
keypoints_csv.shape

(70, 6)

In [ ]:
keypoints_csv.head()

,key_point_id,key_point,topic,stance,key_point_given,precision_score
560,kp_0_10,Child vaccinations are necessary for the prote...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.625
558,kp_0_10,Child vaccinations are necessary for the prote...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,0.600
1074,kp_0_10,"Child vaccines can be given to anyone, but sho...",Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.500
824,kp_0_10,Child vaccines prevent diseases in the child,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.500
695,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.500


In [ ]:
keypoints_csv= keypoints_csv.drop(['key_point_given', 'precision_score'], axis = 1) 
keypoints_csv.head()

,key_point_id,key_point,topic,stance
560,kp_0_10,Child vaccinations are necessary for the prote...,Routine child vaccinations should be mandatory,1
558,kp_0_10,Child vaccinations are necessary for the prote...,Routine child vaccinations should be mandatory,1
1074,kp_0_10,"Child vaccines can be given to anyone, but sho...",Routine child vaccinations should be mandatory,1
824,kp_0_10,Child vaccines prevent diseases in the child,Routine child vaccinations should be mandatory,1
695,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1


In [ ]:
keypoints_csv.shape

(70, 4)

In [ ]:
keypoints_csv = keypoints_csv.drop_duplicates()
keypoints_csv.shape

(36, 4)

#### Confirming whether we have atleast 5 keypoints belonging to each +1 and -1 stance

In [ ]:
df = keypoints_csv.groupby("topic")

In [ ]:
df.get_group('Routine child vaccinations should be mandatory')['stance'].value_counts()

-1    9
 1    4
Name: stance, dtype: int64

In [ ]:
df.get_group('Social media platforms should be regulated by the government')['stance'].value_counts()

 1    4
-1    2
Name: stance, dtype: int64

In [ ]:
df.get_group('The USA is a good country to live in')['stance'].value_counts()

 1    9
-1    8
Name: stance, dtype: int64

## Saving the key_points.csv file

In [ ]:
keypoints_csv.to_csv('/content/drive/MyDrive/EMNLP_folder_4/gen_key_points_predictions/6_epochs_track_2_key_points.csv')